In [1]:
import tensorflow as tf
import numpy as np
from scipy.optimize import linear_sum_assignment

def box_cxcywh_to_xyxy(boxes_raw):
    """ 
    Helper function used to convert raw box output to the 
    top left and bottom right point of the box.
    
    Params:
    - boxes_raw: Tensor of shape (N, 4) containing N boxes in (center x, center y, width, height) format.

    Returns:
    - box_coords: Tensor of shape (N, 4) containing N boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.
    """
    cx, cy, w, h = tf.unstack(boxes_raw, axis=1)

    x_topleft = cx - 0.5 * w
    y_topleft = cy - 0.5 * h
    x_bottomright = cx + 0.5 * w
    y_bottomright = cy + 0.5 * h
    
    box_coords = tf.stack([x_topleft, y_topleft, x_bottomright, y_bottomright], axis=1)
    
    return box_coords

def box_area(boxes):
    """ 
    Helper function used to calculate a bboxes' area given two corners.
    
    Params:
    - boxes: Tensor of shape (N, 4) containing N bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.

    Returns:
    - areas: Tensor of shape (N,) containing the area of each bounding box.
    """
    xmin, ymin, xmax, ymax = tf.split(boxes, 4, axis=1)
    
    # Compute the width and height of each bounding box
    width = tf.squeeze(xmax - xmin, axis=1)
    height = tf.squeeze(ymax - ymin, axis=1)
    
    # Compute the area of each bounding box
    areas = width * height
    
    return areas


def box_iou(box_preds, box_trues):
    """
    Compute the intersection over union (IoU) between pairs of bounding boxes with matching indices.

    Args:
    - box_preds: Tensor of shape (N, 4) containing N predicted bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.
    - box_trues: Tensor of shape (N, 4) containing N ground truth bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.

    Returns:
    - iou: Tensor of shape (N,) containing the IoU between each pair of bounding boxes with matching indices.
    """
    # Compute areas of bounding boxes
    area_preds = box_area(box_preds)
    area_trues = box_area(box_trues)

    # Compute intersection coordinates
    xmin_inter = tf.maximum(box_preds[:, 0], box_trues[:, 0])
    ymin_inter = tf.maximum(box_preds[:, 1], box_trues[:, 1])
    xmax_inter = tf.minimum(box_preds[:, 2], box_trues[:, 2])
    ymax_inter = tf.minimum(box_preds[:, 3], box_trues[:, 3])

    # Compute intersection areas
    width_inter = tf.maximum(0.0, xmax_inter - xmin_inter)
    height_inter = tf.maximum(0.0, ymax_inter - ymin_inter)
    area_inter = width_inter * height_inter

    # Compute union areas
    area_union = area_preds + area_trues - area_inter

    # Compute IoU
    iou = area_inter / area_union

    return iou, area_union

In [26]:
def giou(box_preds, box_trues):
    """
    Calculate the Generalized IoU (GIoU) between all pairs of bounding boxes using TensorFlow.

    Args:
    - box_preds: Tensor of shape (N, 4) containing N bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.
    - box_trues: Tensor of shape (M, 4) containing M bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.

    Returns:
    - giou: Tensor of shape (N, M) containing the Generalized IoU (GIoU) between all pairs of bounding boxes.
    """
    # Expand dimensions to allow broadcasting
    box_preds_expanded = tf.expand_dims(box_preds, axis=1)  # shape: (N, 1, 4)
    box_trues_expanded = tf.expand_dims(box_trues, axis=0)  # shape: (1, M, 4)

    # Compute intersection over union (IoU) and union
    iou = tf.maximum(0.0, tf.minimum(box_preds_expanded[:, :, 2:], box_trues_expanded[:, :, 2:]) - tf.maximum(box_preds_expanded[:, :, :2], box_trues_expanded[:, :, :2]))
    intersection = tf.reduce_prod(iou, axis=-1)  # shape: (N, M)
    area_preds = tf.reduce_prod(box_preds_expanded[:, :, 2:] - box_preds_expanded[:, :, :2], axis=-1)  # shape: (N, 1)
    area_trues = tf.reduce_prod(box_trues_expanded[:, :, 2:] - box_trues_expanded[:, :, :2], axis=-1)  # shape: (1, M)
    union = area_preds + area_trues - intersection  # shape: (N, M)

    # Compute enclosing box (minimum area box that encloses both boxes)
    enclose_min = tf.minimum(box_preds_expanded[:, :, :2], box_trues_expanded[:, :, :2])  # shape: (N, M, 2)
    enclose_max = tf.maximum(box_preds_expanded[:, :, 2:], box_trues_expanded[:, :, 2:])  # shape: (N, M, 2)
    enclose_size = tf.maximum(enclose_max - enclose_min, 0.0)  # shape: (N, M, 2)
    enclose_area = tf.reduce_prod(enclose_size, axis=-1)  # shape: (N, M)

    # Compute Generalized IoU (GIoU)
    giou = intersection / tf.maximum(union, 1e-10) - (enclose_area - union) / tf.maximum(enclose_area, 1e-10)

    return giou

def huber_loss(box_preds, box_trues):
    """
    Compute the Huber loss (smooth-L1) for bounding boxes.

    Parameters:
    - box_preds: Predicted bounding boxes of shape (N, 4) representing (xmin, ymin, xmax, ymax).
    - box_trues: Target bounding boxes of shape (N, 4) representing (xmin, ymin, xmax, ymax).

    Returns:
    - huber_loss: Scalar tensor representing the Huber error (smooth-L1) between the predicted and target bounding boxes.
    """
    loss_fn = tf.keras.losses.Huber()
    huber_loss = loss_fn(box_trues, box_preds)
    
    return huber_loss

def l1_loss(box_preds, box_trues):
    """
    Calculate the L1 loss between all pairs of bounding boxes using TensorFlow.

    Args:
    - box_preds: Tensor of shape (N, 4) containing N bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.
    - box_trues: Tensor of shape (M, 4) containing M bounding boxes in (x_topleft, y_topleft, x_bottomright, y_bottomright) format.

    Returns:
    - loss: Tensor of shape (N, M) containing the L1 loss between all pairs of bounding boxes.
    """
    # Expand dimensions to allow broadcasting
    box_preds_expanded = tf.expand_dims(box_preds, axis=1)  # shape: (N, 1, 4)
    box_trues_expanded = tf.expand_dims(box_trues, axis=0)  # shape: (1, M, 4)

    # Compute L1 loss
    loss = tf.reduce_sum(tf.abs(box_preds_expanded - box_trues_expanded), axis=-1)  # shape: (N, M)

    return loss


def crossentropy_loss(class_preds, class_trues):
    """
    Calculate the cross-entropy loss between all pairs of predicted and true classes using TensorFlow.

    Args:
    - class_preds: Tensor of shape (N, C) containing N sets of class predictions, where C is the number of classes.
    - class_trues: Tensor of shape (M, C) containing M sets of true class labels, where C is the number of classes.

    Returns:
    - loss: Tensor of shape (N, M) containing the cross-entropy loss between all pairs of predicted and true classes.
    """
    # Ensure class_preds are probabilities (apply softmax if they are logits)
    class_preds = tf.nn.softmax(class_preds, axis=1)
    
    # Expand dimensions to allow pairwise computation
    expanded_preds = tf.expand_dims(class_preds, 1)  # Shape becomes (N, 1, C)
    expanded_trues = tf.expand_dims(class_trues, 0)  # Shape becomes (1, M, C)

    # Compute cross-entropy loss pairwise
    loss_matrix = -tf.reduce_sum(expanded_trues * tf.math.log(expanded_preds + 1e-9), axis=2)  # Shape becomes (N, M)
    
    return loss_matrix


In [3]:
def calculate_loss(box_preds, box_trues):
    pass

In [25]:
def tests():
    box_preds = tf.constant([[1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0], [6.0, 4.4, 2.9, 2.0]])
    box_trues = tf.constant([[3.0, 4.0, 7.0, 8.0], [4.0, 5.0, 8.0, 9.0], [6.0, 4.4, 2.9, 2.0]])

    class_preds = tf.constant([[0.4, 0.6], [0.8, 0.2], [0.25, 0.75]])
    class_trues = tf.constant([[0.0, 1.0], [1.0, 0.0], [1.0, 0.0]])
    
    iou_test = box_iou(box_preds, box_trues)
    print(iou_test[0].numpy()) # printing ious
    print(iou_test[1].numpy()) # printing unions (used for gIoU)
    assert(iou_test[0][0] == 1/7)
    assert(iou_test[0][1] == 1/7) 

    giou_test = giou(box_preds, box_trues)
    print(giou_test)
    assert(tf.round(giou_test[0][0], 5) == tf.round(-5/63, 5))

    huber_test = huber_loss(box_preds, box_trues)
    print(huber_test)
    assert(huber_test == 1)

    l1_test = l1_loss(box_preds, box_trues)
    print(l1_test)

    ce_test = crossentropy_loss(class_preds, class_trues)
    print("ce test", ce_test)

tests()

[0.14285715 0.14285715 0.        ]
[28.   28.   14.88]
tf.Tensor(
[[-7.9365075e-02 -3.3508888e-01  4.6500003e-01]
 [ 3.1130433e-01 -7.9365075e-02  4.6500003e-01]
 [ 4.6500003e-01  2.7391309e-01  1.4880000e+11]], shape=(3, 3), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(
[[ 8.       12.       13.5     ]
 [ 4.        8.       13.5     ]
 [13.5      14.700001  0.      ]], shape=(3, 3), dtype=float32)
ce test tf.Tensor(
[[0.5981388  0.7981389  0.7981389 ]
 [1.037488   0.437488   0.437488  ]
 [0.47407696 0.974077   0.974077  ]], shape=(3, 3), dtype=float32)


In [5]:
def np_tf_linear_sum_assignment(matrix):

    indices = linear_sum_assignment(matrix)
    target_indices = indices[0]
    pred_indices = indices[1]

    #print(matrix.shape, target_indices, pred_indices)

    target_selector = np.zeros(matrix.shape[0])
    target_selector[target_indices] = 1
    target_selector = target_selector.astype(np.bool)

    pred_selector = np.zeros(matrix.shape[1])
    pred_selector[pred_indices] = 1
    pred_selector = pred_selector.astype(np.bool)

    #print('target_indices', target_indices)
    #print("pred_indices", pred_indices)

    return [target_indices, pred_indices, target_selector, pred_selector]

In [40]:
def np_tf_linear_sum_assignment(cost_matrix):
    # This is a placeholder for the actual implementation which you would need to adjust
    import numpy as np
    from scipy.optimize import linear_sum_assignment

    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    target_selector = np.zeros(cost_matrix.shape[0], dtype=np.bool_)  # Correcting dtype here
    target_selector[row_ind] = True
    pred_selector = np.zeros(cost_matrix.shape[1], dtype=np.bool_)  # Correcting dtype here
    pred_selector[col_ind] = True

    return row_ind, col_ind, target_selector, pred_selector


def hungarian_matching(t_bbox, t_class, p_bbox, p_class, fcost_class=1, fcost_bbox=5, fcost_giou=2) -> tuple:
    # Convert from [xc, yc, w, h] to [xmin, ymin, xmax, ymax]
    # p_bbox_xy = bbox.xcycwh_to_xy_min_xy_max(p_bbox)
    # t_bbox_xy = bbox.xcycwh_to_xy_min_xy_max(t_bbox)

    # Classification cost for the Hungarian algorithm
    # On each prediction, we select the prob of the expected class
    cost_class = crossentropy_loss(p_class, t_class)

    # L1 cost for the Hungarian algorithm
    cost_bbox = l1_loss(p_bbox, t_bbox)

    # Generalized IOU
    cost_giou = -giou(p_bbox, t_bbox)

    # Final Hungarian cost matrix
    cost_matrix = fcost_bbox * cost_bbox + fcost_class * cost_class + fcost_giou * cost_giou

    print("cost", cost_matrix)

    # np_tf_linear_sum_assignment should be defined elsewhere in your code
    selectors = tf.numpy_function(np_tf_linear_sum_assignment, [cost_matrix], [tf.float64, tf.float64, np.bool_, np.bool_] )
    target_indices = selectors[0]
    pred_indices = selectors[1]
    target_selector = selectors[2]
    pred_selector = selectors[3]

    return pred_indices, target_indices, pred_selector, target_selector, t_bbox, t_class

# Example usage
box_preds = tf.constant([[1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0], [6.0, 4.4, 2.9, 2.0]])
box_trues = tf.constant([[3.0, 4.0, 7.0, 8.0], [4.0, 5.0, 8.0, 9.0], [6.0, 4.4, 2.9, 2.0]])

class_preds = tf.constant([[0.4, 0.6], [0.8, 0.2], [0.25, 0.75]])
class_trues = tf.constant([[0.0, 1.0], [1.0, 0.0], [1.0, 0.0]])

print("HUNGARY", hungarian_matching(box_trues, class_trues, box_preds, class_preds))

box_preds_same = tf.constant([[6.0, 4.4, 2.9, 2.0], [1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0]])
box_trues_same = tf.constant([[1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0], [6.0, 4.4, 2.9, 2.0]])

class_preds_same = tf.constant([[0.0, 1.0], [1.0, 0.0], [1.0, 0.0]])
class_trues_same = tf.constant([[0.0, 1.0], [1.0, 0.0], [1.0, 0.0]])

print("HUNGARY2", hungarian_matching(box_trues_same, class_trues_same, box_preds_same, class_preds_same))


cost tf.Tensor(
[[ 4.0756866e+01  6.1468315e+01  6.7368141e+01]
 [ 2.0414879e+01  4.0596218e+01  6.7007484e+01]
 [ 6.7044075e+01  7.3926247e+01 -2.9759999e+11]], shape=(3, 3), dtype=float32)
HUNGARY (<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 1, 2])>, <tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 1, 2])>, <tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True,  True])>, <tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True,  True])>, <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[3. , 4. , 7. , 8. ],
       [4. , 5. , 8. , 9. ],
       [6. , 4.4, 2.9, 2. ]], dtype=float32)>, <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)>)
cost tf.Tensor(
[[ 6.6883263e+01  6.7883263e+01 -2.9759999e+11]
 [-6.8673837e-01  1.9690653e+01  6.6883263e+01]
 [ 2.0690653e+01 -1.6867384e+00  6.6883263e+01]], shape=(3, 3), dtype=float32)
HUNGARY2 (<tf.Tensor: shape=(3,), dtype=int64, numpy=array([2, 0, 1])

In [91]:
import numpy as np
import tensorflow as tf
from scipy.optimize import linear_sum_assignment

# Helper Functions
def np_tf_linear_sum_assignment(cost_matrix):
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    target_selector = np.zeros(cost_matrix.shape[0], dtype=bool)
    pred_selector = np.zeros(cost_matrix.shape[1], dtype=bool)
    target_selector[row_ind] = True
    pred_selector[col_ind] = True

    return row_ind, col_ind, target_selector, pred_selector

def giou(box_preds, box_trues):
    """
    Calculate the Generalized IoU (GIoU) between all pairs of bounding boxes using TensorFlow.

    Args:
    - box_preds: Tensor of shape (N, 4) containing N bounding boxes in (xmin, ymin, xmax, ymax) format.
    - box_trues: Tensor of shape (M, 4) containing M bounding boxes in (xmin, ymin, xmax, ymax) format.

    Returns:
    - giou: Tensor of shape (N, M) containing the Generalized IoU (GIoU) between all pairs of bounding boxes.
    """
    box_preds_expanded = tf.expand_dims(box_preds, axis=1)  # shape: (N, 1, 4)
    box_trues_expanded = tf.expand_dims(box_trues, axis=0)  # shape: (1, M, 4)

    # Calculate intersection
    inter_mins = tf.maximum(box_preds_expanded[:, :, :2], box_trues_expanded[:, :, :2])
    inter_maxs = tf.minimum(box_preds_expanded[:, :, 2:], box_trues_expanded[:, :, 2:])
    intersection = tf.maximum(inter_maxs - inter_mins, 0.0)
    intersection_area = tf.reduce_prod(intersection, axis=-1)

    # Calculate individual box areas
    area_preds = tf.reduce_prod(box_preds_expanded[:, :, 2:] - box_preds_expanded[:, :, :2], axis=-1)
    area_trues = tf.reduce_prod(box_trues_expanded[:, :, 2:] - box_trues_expanded[:, :, :2], axis=-1)

    # Calculate union
    union = area_preds + area_trues - intersection_area
    union = tf.maximum(union, 1e-10)

    # Calculate enclosing box
    enclose_min = tf.minimum(box_preds_expanded[:, :, :2], box_trues_expanded[:, :, :2])
    enclose_max = tf.maximum(box_preds_expanded[:, :, 2:], box_trues_expanded[:, :, 2:])
    enclose_area = tf.reduce_prod(tf.maximum(enclose_max - enclose_min, 0.0), axis=-1)
    enclose_area = tf.maximum(enclose_area, 1e-10)

    # Generalized IoU calculation
    giou = intersection_area / union - (enclose_area - union) / enclose_area

    return giou 


def l1_loss(box_preds, box_trues):
    box_preds_expanded = tf.expand_dims(box_preds, axis=1)
    box_trues_expanded = tf.expand_dims(box_trues, axis=0)
    loss = tf.reduce_sum(tf.abs(box_preds_expanded - box_trues_expanded), axis=-1)
    return loss

def crossentropy_loss(class_preds, class_trues):
    expanded_preds = tf.expand_dims(class_preds, 1)
    expanded_trues = tf.expand_dims(class_trues, 0)
    loss_matrix = -tf.reduce_sum(expanded_trues * tf.math.log(expanded_preds + 1e-9), axis=2)
    return loss_matrix

# Optimized Matching Function
def hungarian_matching(t_bbox, t_class, p_bbox, p_class, fcost_class=1, fcost_bbox=5, fcost_giou=2):
    cost_class = crossentropy_loss(p_class, t_class)
    cost_bbox = l1_loss(p_bbox, t_bbox)
    cost_giou = -giou(p_bbox, t_bbox)

    print(cost_class)
    print(cost_bbox)
    print(cost_giou)

    cost_matrix = fcost_bbox * cost_bbox + fcost_class * cost_class + fcost_giou * cost_giou
    print(cost_matrix)

    selectors = tf.numpy_function(np_tf_linear_sum_assignment, [cost_matrix], [tf.int64, tf.int64, tf.bool, tf.bool])
    target_indices = tf.cast(selectors[0], tf.int32)
    pred_indices = tf.cast(selectors[1], tf.int32)
    target_selector = selectors[2]
    pred_selector = selectors[3]

    matched_t_bbox = tf.gather(t_bbox, target_indices)
    matched_t_class = tf.gather(t_class, target_indices)
    matched_p_bbox = tf.gather(p_bbox, pred_indices)
    matched_p_class = tf.gather(p_class, pred_indices)

    optimized_cost_class = tf.reduce_mean(crossentropy_loss(matched_p_class, matched_t_class))
    optimized_cost_bbox = tf.reduce_mean(l1_loss(matched_p_bbox, matched_t_bbox))
    optimized_cost_giou = tf.reduce_mean(-giou(matched_p_bbox, matched_t_bbox))

    optimized_loss = fcost_bbox * optimized_cost_bbox + fcost_class * optimized_cost_class + fcost_giou * optimized_cost_giou

    print("Optimized Loss:", optimized_loss.numpy())

    return pred_indices, target_indices, pred_selector, target_selector, t_bbox, t_class

box_preds = tf.constant([[1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0], [1.0, 2.0, 5.0, 6.0]], dtype=tf.float32)
box_trues = tf.constant([[2.0, 3.0, 6.0, 7.0], [1.0, 2.0, 5.0, 6.0], [1.0, 2.0, 5.0, 6.0]], dtype=tf.float32)

class_preds = tf.constant([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0]], dtype=tf.float32)
class_trues = tf.constant([[0.0, 1.0], [1.0, 0.0], [1.0, 0.0]], dtype=tf.float32)

# Run the Hungarian matching and calculate optimized loss
print(hungarian_matching(box_trues, class_trues, box_preds, class_preds))


tf.Tensor(
[[20.723267 -0.       -0.      ]
 [-0.       20.723267 20.723267]
 [20.723267 -0.       -0.      ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[4. 0. 0.]
 [0. 4. 4.]
 [4. 0. 0.]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[-0.31130433 -1.         -1.        ]
 [-1.         -0.31130433 -0.31130433]
 [-0.31130433 -1.         -1.        ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[40.10066 -2.      -2.     ]
 [-2.      40.10066 40.10066]
 [40.10066 -2.      -2.     ]], shape=(3, 3), dtype=float32)
Optimized Loss: 16.711403
(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 0, 2], dtype=int32)>, <tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>, <tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True,  True])>, <tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True,  True])>, <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[2., 3., 6., 7.],
       [1., 2., 5., 6.],
       [1., 2., 5., 6.]], dtype=float32)>, <tf.Tensor: shape=(3, 2), 

In [93]:
import numpy as np
import tensorflow as tf
from scipy.optimize import linear_sum_assignment

# Helper Functions
def np_tf_linear_sum_assignment(cost_matrix):
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    target_selector = np.zeros(cost_matrix.shape[0], dtype=bool)
    pred_selector = np.zeros(cost_matrix.shape[1], dtype=bool)
    target_selector[row_ind] = True
    pred_selector[col_ind] = True

    return row_ind, col_ind, target_selector, pred_selector

def giou(box_preds, box_trues):
    """
    Calculate the Generalized IoU (GIoU) between all pairs of bounding boxes using TensorFlow.

    Args:
    - box_preds: Tensor of shape (N, 4) containing N bounding boxes in (xmin, ymin, xmax, ymax) format.
    - box_trues: Tensor of shape (M, 4) containing M bounding boxes in (xmin, ymin, xmax, ymax) format.

    Returns:
    - giou: Tensor of shape (N, M) containing the Generalized IoU (GIoU) between all pairs of bounding boxes.
    """
    box_preds_expanded = tf.expand_dims(box_preds, axis=1)  # shape: (N, 1, 4)
    box_trues_expanded = tf.expand_dims(box_trues, axis=0)  # shape: (1, M, 4)

    # Calculate intersection
    inter_mins = tf.maximum(box_preds_expanded[:, :, :2], box_trues_expanded[:, :, :2])
    inter_maxs = tf.minimum(box_preds_expanded[:, :, 2:], box_trues_expanded[:, :, 2:])
    intersection = tf.maximum(inter_maxs - inter_mins, 0.0)
    intersection_area = tf.reduce_prod(intersection, axis=-1)

    # Calculate individual box areas
    area_preds = tf.reduce_prod(box_preds_expanded[:, :, 2:] - box_preds_expanded[:, :, :2], axis=-1)
    area_trues = tf.reduce_prod(box_trues_expanded[:, :, 2:] - box_trues_expanded[:, :, :2], axis=-1)

    # Calculate union
    union = area_preds + area_trues - intersection_area
    union = tf.maximum(union, 1e-10)

    # Calculate enclosing box
    enclose_min = tf.minimum(box_preds_expanded[:, :, :2], box_trues_expanded[:, :, :2])
    enclose_max = tf.maximum(box_preds_expanded[:, :, 2:], box_trues_expanded[:, :, 2:])
    enclose_area = tf.reduce_prod(tf.maximum(enclose_max - enclose_min, 0.0), axis=-1)
    enclose_area = tf.maximum(enclose_area, 1e-10)

    # Generalized IoU calculation
    giou = intersection_area / union - (enclose_area - union) / enclose_area

    return giou

def l1_loss(box_preds, box_trues):
    box_preds_expanded = tf.expand_dims(box_preds, axis=1)
    box_trues_expanded = tf.expand_dims(box_trues, axis=0)
    loss = tf.reduce_sum(tf.abs(box_preds_expanded - box_trues_expanded), axis=-1)
    return loss

def crossentropy_loss(class_preds, class_trues):
    """
    Calculate cross-entropy loss between predictions and true classes directly from probabilities.
    """
    expanded_preds = tf.expand_dims(class_preds, 1)
    expanded_trues = tf.expand_dims(class_trues, 0)
    loss_matrix = -tf.reduce_sum(expanded_trues * tf.math.log(expanded_preds + 1e-9), axis=2)
    return loss_matrix

# Optimized Matching Function
def hungarian_matching(t_bbox, t_class, p_bbox, p_class, fcost_class=1, fcost_bbox=5, fcost_giou=2):
    cost_class = crossentropy_loss(p_class, t_class)
    cost_bbox = l1_loss(p_bbox, t_bbox)
    cost_giou = -giou(p_bbox, t_bbox)

    print("\nInitial Costs:")
    print("Cross-Entropy Loss Matrix:", cost_class)
    print("L1 Loss Matrix:", cost_bbox)
    print("GIoU Loss Matrix:", cost_giou)

    cost_matrix = fcost_bbox * cost_bbox + fcost_class * cost_class + fcost_giou * cost_giou
    print("\nCombined Cost Matrix:", cost_matrix)

    selectors = tf.numpy_function(np_tf_linear_sum_assignment, [cost_matrix], [tf.int64, tf.int64, tf.bool, tf.bool])
    target_indices = tf.cast(selectors[0], tf.int32)
    pred_indices = tf.cast(selectors[1], tf.int32)
    target_selector = selectors[2]
    pred_selector = selectors[3]

    print("\nHungarian Matching Indices:")
    print("Target Indices:", target_indices)
    print("Prediction Indices:", pred_indices)

    matched_t_bbox = tf.gather(t_bbox, target_indices)
    matched_t_class = tf.gather(t_class, target_indices)
    matched_p_bbox = tf.gather(p_bbox, pred_indices)
    matched_p_class = tf.gather(p_class, pred_indices)

    optimized_cost_class = tf.reduce_mean(crossentropy_loss(matched_p_class, matched_t_class))
    optimized_cost_bbox = tf.reduce_mean(l1_loss(matched_p_bbox, matched_t_bbox))
    optimized_cost_giou = tf.reduce_mean(-giou(matched_p_bbox, matched_t_bbox))

    optimized_loss = fcost_bbox * optimized_cost_bbox + fcost_class * optimized_cost_class + fcost_giou * optimized_cost_giou

    print("\nOptimized Costs:")
    print("Optimized Cross-Entropy Loss:", optimized_cost_class)
    print("Optimized L1 Loss:", optimized_cost_bbox)
    print("Optimized GIoU Loss:", optimized_cost_giou)
    print("Optimized Loss:", optimized_loss.numpy())

    return pred_indices, target_indices, pred_selector, target_selector, t_bbox, t_class

# Test Inputs
box_preds = tf.constant([[1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0], [1.0, 2.0, 5.0, 6.0]], dtype=tf.float32)
box_trues = tf.constant([[2.0, 3.0, 6.0, 7.0], [1.0, 2.0, 5.0, 6.0], [1.0, 2.0, 5.0, 6.0]], dtype=tf.float32)

class_preds = tf.constant([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0]], dtype=tf.float32)
class_trues = tf.constant([[0.0, 1.0], [1.0, 0.0], [1.0, 0.0]], dtype=tf.float32)

# Run the Hungarian matching and calculate optimized loss
hungarian_matching(box_trues, class_trues, box_preds, class_preds)




Initial Costs:
Cross-Entropy Loss Matrix: tf.Tensor(
[[20.723267 -0.       -0.      ]
 [-0.       20.723267 20.723267]
 [20.723267 -0.       -0.      ]], shape=(3, 3), dtype=float32)
L1 Loss Matrix: tf.Tensor(
[[4. 0. 0.]
 [0. 4. 4.]
 [4. 0. 0.]], shape=(3, 3), dtype=float32)
GIoU Loss Matrix: tf.Tensor(
[[-0.31130433 -1.         -1.        ]
 [-1.         -0.31130433 -0.31130433]
 [-0.31130433 -1.         -1.        ]], shape=(3, 3), dtype=float32)

Combined Cost Matrix: tf.Tensor(
[[40.10066 -2.      -2.     ]
 [-2.      40.10066 40.10066]
 [40.10066 -2.      -2.     ]], shape=(3, 3), dtype=float32)

Hungarian Matching Indices:
Target Indices: tf.Tensor([0 1 2], shape=(3,), dtype=int32)
Prediction Indices: tf.Tensor([1 0 2], shape=(3,), dtype=int32)

Optimized Costs:
Optimized Cross-Entropy Loss: tf.Tensor(9.2103405, shape=(), dtype=float32)
Optimized L1 Loss: tf.Tensor(1.7777778, shape=(), dtype=float32)
Optimized GIoU Loss: tf.Tensor(-0.69391304, shape=(), dtype=float32)
Optimized

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 0, 2], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True,  True])>,
 <tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True,  True])>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[2., 3., 6., 7.],
        [1., 2., 5., 6.],
        [1., 2., 5., 6.]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0., 1.],
        [1., 0.],
        [1., 0.]], dtype=float32)>)

In [94]:
def hungarian_matching(t_bbox, t_class, p_bbox, p_class, fcost_class=1, fcost_bbox=5, fcost_giou=2):
    cost_class = crossentropy_loss(p_class, t_class)
    cost_bbox = l1_loss(p_bbox, t_bbox)
    cost_giou = -giou(p_bbox, t_bbox)

    print("\nInitial Costs:")
    print("Cross-Entropy Loss Matrix:", cost_class)
    print("L1 Loss Matrix:", cost_bbox)
    print("GIoU Loss Matrix:", cost_giou)

    cost_matrix = fcost_bbox * cost_bbox + fcost_class * cost_class + fcost_giou * cost_giou
    print("\nCombined Cost Matrix:", cost_matrix)

    selectors = tf.numpy_function(np_tf_linear_sum_assignment, [cost_matrix], [tf.int64, tf.int64, tf.bool, tf.bool])
    target_indices = tf.cast(selectors[0], tf.int32)
    pred_indices = tf.cast(selectors[1], tf.int32)
    target_selector = selectors[2]
    pred_selector = selectors[3]

    print("\nHungarian Matching Indices:")
    print("Target Indices:", target_indices)
    print("Prediction Indices:", pred_indices)

    matched_t_bbox = tf.gather(t_bbox, target_indices)
    matched_t_class = tf.gather(t_class, target_indices)
    matched_p_bbox = tf.gather(p_bbox, pred_indices)
    matched_p_class = tf.gather(p_class, pred_indices)

    print("\nMatched Targets and Predictions:")
    print("Matched Target Bounding Boxes:", matched_t_bbox)
    print("Matched Prediction Bounding Boxes:", matched_p_bbox)
    print("Matched Target Classes:", matched_t_class)
    print("Matched Prediction Classes:", matched_p_class)

    optimized_cost_class = tf.reduce_mean(crossentropy_loss(matched_p_class, matched_t_class))
    optimized_cost_bbox = tf.reduce_mean(l1_loss(matched_p_bbox, matched_t_bbox))
    optimized_cost_giou = tf.reduce_mean(-giou(matched_p_bbox, matched_t_bbox))

    optimized_loss = fcost_bbox * optimized_cost_bbox + fcost_class * optimized_cost_class + fcost_giou * optimized_cost_giou

    print("\nOptimized Costs:")
    print("Optimized Cross-Entropy Loss:", optimized_cost_class)
    print("Optimized L1 Loss:", optimized_cost_bbox)
    print("Optimized GIoU Loss:", optimized_cost_giou)
    print("Optimized Loss:", optimized_loss.numpy())

    return pred_indices, target_indices, pred_selector, target_selector, t_bbox, t_class

# Test Inputs
box_preds = tf.constant([[1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0], [1.0, 2.0, 5.0, 6.0]], dtype=tf.float32)
box_trues = tf.constant([[2.0, 3.0, 6.0, 7.0], [1.0, 2.0, 5.0, 6.0], [1.0, 2.0, 5.0, 6.0]], dtype=tf.float32)

class_preds = tf.constant([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0]], dtype=tf.float32)
class_trues = tf.constant([[0.0, 1.0], [1.0, 0.0], [1.0, 0.0]], dtype=tf.float32)

# Run the Hungarian matching and calculate optimized loss
hungarian_matching(box_trues, class_trues, box_preds, class_preds)


Initial Costs:
Cross-Entropy Loss Matrix: tf.Tensor(
[[20.723267 -0.       -0.      ]
 [-0.       20.723267 20.723267]
 [20.723267 -0.       -0.      ]], shape=(3, 3), dtype=float32)
L1 Loss Matrix: tf.Tensor(
[[4. 0. 0.]
 [0. 4. 4.]
 [4. 0. 0.]], shape=(3, 3), dtype=float32)
GIoU Loss Matrix: tf.Tensor(
[[-0.31130433 -1.         -1.        ]
 [-1.         -0.31130433 -0.31130433]
 [-0.31130433 -1.         -1.        ]], shape=(3, 3), dtype=float32)

Combined Cost Matrix: tf.Tensor(
[[40.10066 -2.      -2.     ]
 [-2.      40.10066 40.10066]
 [40.10066 -2.      -2.     ]], shape=(3, 3), dtype=float32)

Hungarian Matching Indices:
Target Indices: tf.Tensor([0 1 2], shape=(3,), dtype=int32)
Prediction Indices: tf.Tensor([1 0 2], shape=(3,), dtype=int32)

Matched Targets and Predictions:
Matched Target Bounding Boxes: tf.Tensor(
[[2. 3. 6. 7.]
 [1. 2. 5. 6.]
 [1. 2. 5. 6.]], shape=(3, 4), dtype=float32)
Matched Prediction Bounding Boxes: tf.Tensor(
[[2. 3. 6. 7.]
 [1. 2. 5. 6.]
 [1. 2. 5

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 0, 2], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>,
 <tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True,  True])>,
 <tf.Tensor: shape=(3,), dtype=bool, numpy=array([ True,  True,  True])>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[2., 3., 6., 7.],
        [1., 2., 5., 6.],
        [1., 2., 5., 6.]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0., 1.],
        [1., 0.],
        [1., 0.]], dtype=float32)>)

In [10]:
import numpy as np
import tensorflow as tf
from scipy.optimize import linear_sum_assignment

# Helper Functions
def np_tf_linear_sum_assignment(cost_matrix):
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    target_selector = np.zeros(cost_matrix.shape[0], dtype=bool)
    pred_selector = np.zeros(cost_matrix.shape[1], dtype=bool)
    target_selector[row_ind] = True
    pred_selector[col_ind] = True

    return row_ind, col_ind, target_selector, pred_selector

def giou(box_preds, box_trues):
    """
    Calculate the Generalized IoU (GIoU) between all pairs of bounding boxes using TensorFlow.

    Args:
    - box_preds: Tensor of shape (N, 4) containing N bounding boxes in (xmin, ymin, xmax, ymax) format.
    - box_trues: Tensor of shape (M, 4) containing M bounding boxes in (xmin, ymin, xmax, ymax) format.

    Returns:
    - giou: Tensor of shape (N, M) containing the Generalized IoU (GIoU) between all pairs of bounding boxes.
    """
    box_preds_expanded = tf.expand_dims(box_preds, axis=1)  # shape: (N, 1, 4)
    box_trues_expanded = tf.expand_dims(box_trues, axis=0)  # shape: (1, M, 4)

    # Calculate intersection
    inter_mins = tf.maximum(box_preds_expanded[:, :, :2], box_trues_expanded[:, :, :2])
    inter_maxs = tf.minimum(box_preds_expanded[:, :, 2:], box_trues_expanded[:, :, 2:])
    intersection = tf.maximum(inter_maxs - inter_mins, 0.0)
    intersection_area = tf.reduce_prod(intersection, axis=-1)

    # Calculate individual box areas
    area_preds = tf.reduce_prod(box_preds_expanded[:, :, 2:] - box_preds_expanded[:, :, :2], axis=-1)
    area_trues = tf.reduce_prod(box_trues_expanded[:, :, 2:] - box_trues_expanded[:, :, :2], axis=-1)

    # Calculate union
    union = area_preds + area_trues - intersection_area
    union = tf.maximum(union, 1e-10)

    # Calculate enclosing box
    enclose_min = tf.minimum(box_preds_expanded[:, :, :2], box_trues_expanded[:, :, :2])
    enclose_max = tf.maximum(box_preds_expanded[:, :, 2:], box_trues_expanded[:, :, 2:])
    enclose_area = tf.reduce_prod(tf.maximum(enclose_max - enclose_min, 0.0), axis=-1)
    enclose_area = tf.maximum(enclose_area, 1e-10)

    # Generalized IoU calculation
    giou = intersection_area / union - (enclose_area - union) / enclose_area

    return giou

def l1_loss(box_preds, box_trues):
    box_preds_expanded = tf.expand_dims(box_preds, axis=1)
    box_trues_expanded = tf.expand_dims(box_trues, axis=0)
    loss = tf.reduce_sum(tf.abs(box_preds_expanded - box_trues_expanded), axis=-1)
    return loss

def crossentropy_loss(class_preds, class_trues):
    """
    Calculate cross-entropy loss between predictions and true classes directly from probabilities.
    """
    expanded_preds = tf.expand_dims(class_preds, 1)
    expanded_trues = tf.expand_dims(class_trues, 0)
    loss_matrix = -tf.reduce_sum(expanded_trues * tf.math.log(expanded_preds + 1e-9), axis=2)

    # Check for perfect matches and set their loss to zero
    is_perfect_match = tf.reduce_all(tf.equal(expanded_preds, expanded_trues), axis=2)
    loss_matrix = tf.where(is_perfect_match, 0.0, loss_matrix)

    return loss_matrix

# Optimized Matching Function
def hungarian_matching(t_bbox, t_class, p_bbox, p_class, fcost_class=1, fcost_bbox=5, fcost_giou=2):
    cost_class = crossentropy_loss(p_class, t_class)
    cost_bbox = l1_loss(p_bbox, t_bbox)
    cost_giou = -giou(p_bbox, t_bbox)

    print("\nInitial Costs:")
    print("Cross-Entropy Loss Matrix:", cost_class)
    print("L1 Loss Matrix:", cost_bbox)
    print("GIoU Loss Matrix:", cost_giou)

    cost_matrix = fcost_bbox * cost_bbox + fcost_class * cost_class + fcost_giou * cost_giou
    print("\nCombined Cost Matrix:", cost_matrix)

    selectors = tf.numpy_function(np_tf_linear_sum_assignment, [cost_matrix], [tf.int64, tf.int64, tf.bool, tf.bool])
    target_indices = tf.cast(selectors[0], tf.int32)
    pred_indices = tf.cast(selectors[1], tf.int32)
    target_selector = selectors[2]
    pred_selector = selectors[3]

    optimized_cost = 0
    for i in range (len(target_indices)):
        optimized_cost += cost_matrix[target_indices[i]][pred_indices[i]]

    return optimized_cost, pred_indices, target_indices, pred_selector, target_selector, t_bbox, t_class

# Test Inputs
box_preds = tf.constant([[1.0, 2.0, 5.0, 6.0], [2.0, 3.0, 6.0, 7.0], [1.0, 2.0, 5.0, 6.0]], dtype=tf.float32)
box_trues = tf.constant([[2.0, 3.0, 6.0, 7.0], [1.0, 2.0, 5.0, 6.0], [1.0, 2.0, 5.0, 6.0]], dtype=tf.float32)

class_preds = tf.constant([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0]], dtype=tf.float32)
class_trues = tf.constant([[0.0, 1.0], [1.0, 0.0], [1.0, 0.0]], dtype=tf.float32)

# Run the Hungarian matching and calculate optimized loss
hungarian_matching(box_trues, class_trues, box_preds, class_preds)



Initial Costs:
Cross-Entropy Loss Matrix: tf.Tensor(
[[20.723265  0.        0.      ]
 [ 0.       20.723265 20.723265]
 [20.723267  0.        0.      ]], shape=(3, 3), dtype=float32)
L1 Loss Matrix: tf.Tensor(
[[4. 0. 0.]
 [0. 4. 4.]
 [4. 0. 0.]], shape=(3, 3), dtype=float32)
GIoU Loss Matrix: tf.Tensor(
[[-0.31130433 -1.         -1.        ]
 [-1.         -0.31130433 -0.31130433]
 [-0.31130433 -1.         -1.        ]], shape=(3, 3), dtype=float32)

Combined Cost Matrix: tf.Tensor(
[[40.10066 -2.      -2.     ]
 [-2.      40.10066 40.10066]
 [40.10066 -2.      -2.     ]], shape=(3, 3), dtype=float32)
tf.Tensor(-2.0, shape=(), dtype=float32)
tf.Tensor(-2.0, shape=(), dtype=float32)
tf.Tensor(-2.0, shape=(), dtype=float32)
tf.Tensor(-6.0, shape=(), dtype=float32)
